<a href="https://colab.research.google.com/github/KorneliuszDyszczynski/ColorizingPhotosNeuralNetwork/blob/master/ColorizingImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r ../Humans2/*

rm: cannot remove '../Humans2/*': No such file or directory


In [2]:

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('ashwingupta3012/human-faces')

In [1]:
import keras
import numpy as np
import keras.preprocessing.image
from keras.preprocessing.image import img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
import tensorflow as tf

from PIL import Image
import os, sys
from matplotlib.pyplot import imsave

In [ ]:
!unzip human-faces.zip

In [2]:
os.chdir('/content/Humans/') 
path='/content/Humans/'
path2='/content/Humans2/'
path3='/content/'
COUNT =0
w = 200
h = 200
stop=10;

# for f in os.listdir():
#     f_name, f_ext = os.path.splitext(f)
#     f_name = "face" + str(COUNT) 
#     COUNT = COUNT + 1
#     new_name = '{}{}'.format(f_name, f_ext)
#     os.rename(f, new_name)

for f in os.listdir():
    COUNT = COUNT + 1
    if(COUNT<stop):
      im = Image.open(path+f)
      im = im.resize((w, h))
      im = im.convert('RGB')
      im.save(path2+"resized"+f)

os.chdir('/content/Humans2/') 

images = [Image.open(f) for f in os.listdir()]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

# x_offset = 0
# for im in images:
#   new_im.paste(im, (x_offset,0))
#   x_offset += im.size[0]

# new_im.save(path3+'test.jpg')


# image = img_to_array(load_img(path3+'test.jpg'))
# image = np.array(image, dtype=float)

# Get images
X = []
for filename in os.listdir('/content/Humans2/'):
    X.append(img_to_array(load_img('/content/Humans2/'+filename)))
X = np.array(X, dtype=float)
# Set up training and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain


In [64]:
datagen = keras.preprocessing.image.ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

batch_size = 50
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 100
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [7]:
# X = rgb2lab(1.0/255*image)[:,:,0]
# Y = rgb2lab(1.0/255*image)[:,:,1:]
# Y = Y / 128
# witdthvar=(stop-1)*200;
# X = X.reshape(1, 200, witdthvar, 1)
# Y = Y.reshape(1, 200, witdthvar, 2)

In [65]:
model = Sequential()
model.add(InputLayer(input_shape=(200, 200, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

model.compile(optimizer='rmsprop',loss='mse')

In [74]:
model.fit_generator(image_a_b_gen(batch_size), steps_per_epoch=40, epochs=2)
# print(model.evaluate(X, Y, batch_size=5000))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
40/40 [==============================] - 6s 149ms/step - loss: 0.0089
Epoch 2/2
40/40 [==============================] - 6s 149ms/step - loss: 0.0064


In [75]:
color_me = []
# for filename in os.listdir('/Test/'):
#         color_me.append(img_to_array(load_img('/Test/'+filename)))
color_me.append(img_to_array(load_img('/Test/woman_smoll.jpg')))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


In [76]:
output = model.predict(color_me)
output = output * 128

for i in range(len(output)):
        cur = np.zeros((200, 200, 3))
        cur[:,:,0] = color_me[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave(path3+"img_result.png", lab2rgb(cur))


# canvas = np.zeros((200, 200, 3))
# canvas[:,:,0] = color_me[0][:,:,0]
# canvas[:,:,1:] = output[0]
# imsave(path3+"img_result.png", lab2rgb(canvas))
# imsave(path3+"img_gray_scale.png", rgb2gray(lab2rgb(canvas)))
